<a href="https://colab.research.google.com/github/happyhoo97/Skin-Cancer-Dectection-Application/blob/main/Cancer_Dectection_MobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras

In [5]:
base_dir = '/content/drive/Shareddrives/[서비스데사] 피부암/Dataset_mel_nv'

In [6]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = 'training'
)

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = 'validation'
)

Found 6255 images belonging to 2 classes.
Found 1563 images belonging to 2 classes.


In [7]:
print(train_generator.class_indices)
labels = '\n'.join(sorted(
    train_generator.class_indices.keys()))
with open('labels.txt', 'w') as f:
  f.write(labels)

{'mel_image': 0, 'nv_image': 1}


In [8]:
# 보다 나은 성능을 위해, 전이 학습 수행

IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE) + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

9420800/9406464 [==============================] - 0s 0us/step


In [9]:
base_model.trainable = False
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(2, activation='softmax')

])

In [10]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
epochs = 1
history = model.fit(
    train_generator,
    epochs = epochs,
    validation_data = val_generator,
    verbose = 1
)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("model.tflite", "wb").write(tflite_model)